In [ ]:
from data_reader import Fnc1Reader
import pandas as pd
from simpletransformers.classification import ClassificationModel


def fold_to_transf_input(fold: pd.DataFrame, reader: Fnc1Reader):
    assert fold.columns.tolist() == ['Headline', 'Body ID', 'Stance', 'Label']
    result = fold.copy()
    result['Body ID'] = result['Body ID'].map(reader.get_body_train)
    del result['Stance']
    result = result.rename(columns={'Headline': 'text_a', 'Body ID': 'text_b', 'Label': 'labels'})
    return result


if __name__ == '__main__':
    reader = Fnc1Reader('data/fnc-1/')
    fold = reader.kfold(10)[0]
    data = fold_to_transf_input(fold, reader)


    model = ClassificationModel('albert', 'albert-base-v2', num_labels=4, use_cuda=False, args={
        'learning_rate':3e-5,
        'num_train_epochs': 5,
        'reprocess_input_data': True,
        'overwrite_output_dir': True,
        'process_count': 10,
        'train_batch_size': 4,
        'eval_batch_size': 4,
        'max_seq_length': 512,
        'fp16': False,
    })

    model.train_model(data)